# Setup

In [7]:
%cd /kaggle/working/
%rm -rf mlproduction
!git clone https://github.com/daoanhkhoa123/mlproduction.git
%cd /kaggle/working/mlproduction
!ls

/kaggle/working
Cloning into 'mlproduction'...
remote: Enumerating objects: 228, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 228 (delta 124), reused 172 (delta 69), pack-reused 0 (from 0)
Receiving objects: 100% (228/228), 3.46 MiB | 18.16 MiB/s, done.
Resolving deltas: 100% (124/124), done.
/kaggle/working/mlproduction
main.py  pyproject.toml  README.md  runtime  scripts  sqls  src  uv.lock


In [8]:
!chmod +x scripts/*.sh
!scripts/cloud_setup.sh

Using CPython 3.12.12
Creating virtual environment at: .venv
Resolved 72 packages in 1ms
░░░░░░░░░░░░░░░░░░░░ [0/66] Installing wheels...                                warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 66 packages in 4.36s                              
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.2
 + aiosignal==1.4.0
 + annotated-types==0.7.0
 + attrs==25.4.0
 + certifi==2025.11.12
 + charset-normalizer==3.4.4
 + filelock==3.20.0
 + frozenlist==1.8.0
 + fsspec==2025.12.0
 + hf-xet==1.2.0
 + huggingface-hub==0.36.0
 + idna==3.11
 + isort==7.0.0
 + jinja2==3.1.6
 + joblib==1.5.3
 + lightning==2.6.0
 + lightning-utilities==0.15.2
 + markupsafe==3.0.2
 + mpmath==1.3.0
 + multidict==6.7.0
 + n

In [10]:
!cat scripts/cloud_setup.sh

set -e

# Install uv and sync basic dependencies

pip install --upgrade uv
uv sync

# Install optional dependencies

ENV_FILE="$(cd "$(dirname "$0")/.." && pwd)/.env"
echo "ENV_FILE=$ENV_FILE"

if [ -f "$ENV_FILE" ]; then
    CLOUD_DEPS=$(grep '^CLOUD_DEPENDENCIES' "$ENV_FILE" | cut -d= -f2- | xargs)
else
    echo ".env file not found!"
    exit 1
fi

IFS=',' read -ra DEP_ARRAY <<< "$CLOUD_DEPS"

DEPS=""
for dep in "${DEP_ARRAY[@]}"; do
    dep=$(echo "$dep" | xargs)

    echo "Runing: uv add $dep"
    uv add $dep
done

# Set up custom ipynb kernel

PROJECT_NAME="mlproduction"
VENV_DIR=".venv"
KERNEL_NAME="mlproduction"
DISPLAY_NAME="Python (mlproduction)"

uv run python -m ipykernel install \
    --user \
    --name "${KERNEL_NAME}" \
    --display-name "${DISPLAY_NAME}"

echo "Kernel registered successfully"
echo "Available kernels:"
jupyter kernelspec list

# Path environment fix

KERNEL_DIR="$(jupyter kernelspec list --json | python -c "
import json,sys
data=json.load(sys.stdin)
pr

# Run

In [1]:
import mlflow

In [17]:
!uv run which python

/kaggle/working/mlproduction/.venv/bin/python


In [4]:
import pandas as pd
df = pd.read_csv("data/data.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/data.csv'

NameError: name 'tokenizer' is not defined

In [13]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModelForMaskedLM.from_pretrained("vinai/phobert-base")

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [34]:
import torch

text = "(attended): KhoaDACE190399 had attended this activity / Đào Anh Khoa đã tham gia hoạt động này (absent): KhoaDACE190399 had NOT attended this activity / Đào Anh Khoa đã vắng mặt buổi này (-): no data was given / chưa có dữ liệu"

tokens = tokenizer(text, return_tensors="pt")
print(tokens)

with torch.no_grad():
    output= model(**tokens)

output["logits"].shape

{'input_ids': tensor([[    0, 14157,  3148,  1187,  6374,  3995, 37272,    27,  8044,  1850,
         16311,  9594,  4917, 12627,  4424, 30498,  3148,  7104, 39573, 22304,
          3065, 20712,  8233,    95,  5483,   157,  3720,    14,  7520,  3931,
         11297,  2697,    23, 14157,  3377,  1302,  6502, 37272,    27,  8044,
          1850, 16311,  9594,  4917, 12627,  4424, 30498, 13986,  1085,  3148,
          7104, 39573, 22304,  3065, 20712,  8233,    95,  5483,   157,  3720,
            14,  3543,   373,   391,    23, 14157,   770, 37272,    27,  4397,
         16819, 26431,  2667,  1946,    95,   102,    10,  7015,  1934,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 

torch.Size([1, 80, 64001])